# **Création des images de la base de donnée euroSAT**

**Source :** Les images sont téléchargé depuis le répertoire Github [EuroSAT: Land Use and Land Cover Classification with Sentinel-2 ](https://github.com/phelber/eurosat)  
  
Les **papiers d'origine** depuis lesquelles provient la base geo-referencée euroSAT sont :
- [1] Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.  

- [2] Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.

### **Construction d'une base de donnée**  
Les informations contenues dans cette base seront :
- le nom donné à l'image
- le path vers l'image
- son label

In [2]:
import os
import pandas as pd
import numpy as np

*Préalable* : Durant ce notebook, nous supposerons que le dossier contenant les données téléchargées s'appelle *euroSAT_dataset* et est stocké dans le dossier *euroSAT*.

In [3]:
DATA_DIR = "../euroSAT_dataset"

Récupération des labels de la base.

*Remarque* : Les valeurs *".ipynb_checkpoints"* et *"image_label.csv"* ne devraient pas être présentes, elles figurent dans la sortie de la cellule car celle-ci a été relancée plus tard dans la création du notebook. Nous ne pouvons plus relancer cette cellule car les images seront déplacées par la suite, ignorez seulement ces deux résultats.

In [135]:
labels = os.listdir(DATA_DIR)
for i, label in enumerate(labels):
    print(str(i + 1) + ". " + label)

1. PermanentCrop
2. Pasture
3. Industrial
4. HerbaceousVegetation
5. AnnualCrop
6. Residential
7. Highway
8. River
9. SeaLake
10. .ipynb_checkpoints
11. Forest
12. image_label.csv


Création du chemin vers les dossiers contenant les images pour chaque label.

In [136]:
PermanentCrop_DIR = os.path.join(DATA_DIR, 'PermanentCrop')
Pasture_DIR = os.path.join(DATA_DIR, 'Pasture')
Industrial_DIR = os.path.join(DATA_DIR, 'Industrial')
HerbaceousVegetation_DIR = os.path.join(DATA_DIR, 'HerbaceousVegetation')
AnnualCrop_DIR = os.path.join(DATA_DIR, 'AnnualCrop')
Residential_DIR = os.path.join(DATA_DIR, 'Residential')
Highway_DIR = os.path.join(DATA_DIR, 'Highway')
River_DIR = os.path.join(DATA_DIR, 'River')
SeaLake_DIR = os.path.join(DATA_DIR, 'SeaLake')
Forest_DIR = os.path.join(DATA_DIR, 'Forest')

Récupération du nom des images contenues dans ces derniers.

In [137]:
PermanentCrop_IMG = os.listdir(PermanentCrop_DIR)
Pasture_IMG = os.listdir(Pasture_DIR)
Industrial_IMG = os.listdir(Industrial_DIR)
HerbaceousVegetation_IMG = os.listdir(HerbaceousVegetation_DIR)
AnnualCrop_IMG = os.listdir(AnnualCrop_DIR)
Residential_IMG = os.listdir(Residential_DIR)
Highway_IMG = os.listdir(Highway_DIR)
River_IMG = os.listdir(River_DIR)
SeaLake_IMG = os.listdir(SeaLake_DIR)
Forest_IMG = os.listdir(Forest_DIR)

Aggrégation des résultats.

In [138]:
all_IMG = PermanentCrop_IMG \
            + Pasture_IMG \
            + Industrial_IMG \
            + HerbaceousVegetation_IMG \
            + AnnualCrop_IMG \
            + Residential_IMG \
            + Highway_IMG \
            + River_IMG \
            + SeaLake_IMG \
            + Forest_IMG

In [139]:
print("Nous avons dans chaque dossier le nombre d'image suivant :")
print(len(PermanentCrop_IMG),
len(Pasture_IMG),
len(Industrial_IMG),
len(HerbaceousVegetation_IMG),
len(AnnualCrop_IMG),
len(Residential_IMG),
len(Highway_IMG),
len(River_IMG),
len(SeaLake_IMG),
len(Forest_IMG))
print("Pour un total de {} images.".format(len(all_IMG)))

Nous avons dans chaque dossier le nombre d'image suivant :
2500 2000 2500 3000 3000 3000 2500 2500 3000 3000
Pour un total de 27000 images.


Création d'une liste de dictionnaires contenant le nom de l'image (clé: `image`), le chemin vers cette image (clé: `path`), et son label (clé: `label`). Ce format nous permettra de convetir efficacement cette liste au format `pd.DataFrame`.

In [140]:
liste_dic = [{'image': image,
              'path': os.path.join(DATA_DIR, str.split(image, sep="_")[0], image),
              'label': str.split(image, sep="_")[0]} \
             for image in all_IMG]

Conversion en `DataFrame`.

In [141]:
base = pd.DataFrame(liste_dic)

print("Dimension de notre base de donnée : {}".format(base.shape))
base.head(5)

Dimension de notre base de donnée : (27000, 3)


,image,path,label
0,PermanentCrop_534.jpg,../euroSAT_dataset/PermanentCrop/PermanentCrop...,PermanentCrop
1,PermanentCrop_1297.jpg,../euroSAT_dataset/PermanentCrop/PermanentCrop...,PermanentCrop
2,PermanentCrop_1651.jpg,../euroSAT_dataset/PermanentCrop/PermanentCrop...,PermanentCrop
3,PermanentCrop_755.jpg,../euroSAT_dataset/PermanentCrop/PermanentCrop...,PermanentCrop
4,PermanentCrop_2411.jpg,../euroSAT_dataset/PermanentCrop/PermanentCrop...,PermanentCrop


Sauvegarde de la base de donnée.

In [142]:
base.to_csv(os.path.join(DATA_DIR, 'image_label.csv'), index=False)

### **Séparation des échantillons train / test**

On sauvegarde dans une nouvelle colonne appellée `old_path` le chemin actuelle vers nos images. Le colonne `path` sera ensuite modifiée pour correspondre au future emplacement de l'image et ainsi, répondre à l'agencement des données exigé par le `DataLoader` de `PyTorch`.

In [144]:
base['old_path'] = base['path']

In [146]:
def train_test_split(path):
    """ Associe aleatoirement à chaque image son segment
        (train ou test) et retourne le path correspondant
        au label + segment auquel l'image appartient.
    """
    path_list = str.split(path, sep="/")
    rng = np.random.random()
    if rng < 0.8:
        segment = 'train'
    else:
        segment = 'test'
    path_list = path_list[:2] + [segment] + path_list[2:]
    return ["/".join(path_list), segment]

L'aléa est fixé afin d'asurer la reproductibilité de nos résultats.

In [147]:
np.random.seed(42)

liste_new_path = []
liste_segment = []
for i in range(base.shape[0]):
    resultat = train_test_split(base.loc[i, 'path'])
    liste_new_path.append(resultat[0])
    liste_segment.append(resultat[1])

In [148]:
base['segment'] = liste_segment
base['path'] = liste_new_path

In [149]:
base.head()

,image,path,label,old_path,segment
0,PermanentCrop_534.jpg,../euroSAT_dataset/train/PermanentCrop/Permane...,PermanentCrop,../euroSAT_dataset/PermanentCrop/PermanentCrop...,train
1,PermanentCrop_1297.jpg,../euroSAT_dataset/test/PermanentCrop/Permanen...,PermanentCrop,../euroSAT_dataset/PermanentCrop/PermanentCrop...,test
2,PermanentCrop_1651.jpg,../euroSAT_dataset/train/PermanentCrop/Permane...,PermanentCrop,../euroSAT_dataset/PermanentCrop/PermanentCrop...,train
3,PermanentCrop_755.jpg,../euroSAT_dataset/train/PermanentCrop/Permane...,PermanentCrop,../euroSAT_dataset/PermanentCrop/PermanentCrop...,train
4,PermanentCrop_2411.jpg,../euroSAT_dataset/train/PermanentCrop/Permane...,PermanentCrop,../euroSAT_dataset/PermanentCrop/PermanentCrop...,train


Enregistrement de la base.

In [150]:
base.to_csv(os.path.join(DATA_DIR, 'image_label.csv'), index=False)

### **Modification de l'organisation des images pour le `DataLoader`**

Création des différents dossier

In [ ]:
os.makedirs(os.path.join(DATA_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'test'), exist_ok=True)

In [158]:
for label in labels:
    os.makedirs(os.path.join(DATA_DIR, 'train', label), exist_ok=True)
    os.makedirs(os.path.join(DATA_DIR, 'test', label), exist_ok=True)

Déplacement des images depuis leur emplacement actuelle `old_path` vers leur nouvel emplacement `path`.

In [159]:
for i in range(base.shape[0]):
    os.replace(base.loc[i, 'old_path'], base.loc[i, 'path'])

In [6]:
base = pd.read_csv(os.path.join(DATA_DIR, 'image_label.csv'))

### **Résumé des images contenues dans les échantillons train et test**

In [9]:
print("Nombre d'images par label dans l'échantillon train :")
print(base[base['segment'] == 'train']['label'].value_counts())

print("\nNombre d'images par label dans l'échantillon test :")
print(base[base['segment'] == 'test']['label'].value_counts())

Nombre d'images par label dans l'échantillon train :
HerbaceousVegetation    2444
Forest                  2433
Residential             2389
SeaLake                 2387
AnnualCrop              2382
Highway                 2010
PermanentCrop           1996
Industrial              1985
River                   1965
Pasture                 1613
Name: label, dtype: int64

Nombre d'images par label dans l'échantillon test :
AnnualCrop              618
SeaLake                 613
Residential             611
Forest                  567
HerbaceousVegetation    556
River                   535
Industrial              515
PermanentCrop           504
Highway                 490
Pasture                 387
Name: label, dtype: int64
